<a href="https://www.kaggle.com/code/aahmedehab/tyre-quality-classification-96-test-acc?scriptVersionId=143781495" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# import libraries

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import random
import glob
import cv2
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import  ImageDataGenerator
from keras.layers import Dense,Flatten,BatchNormalization,MaxPooling2D,Conv2D,Input
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras import Model
from sklearn.metrics import classification_report , confusion_matrix

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    print (dirname)

# get images from dataset files


In [ ]:
good = glob.glob(r'/kaggle/input/tyre-quality-classification/Digital images of defective and good condition tyres/good/*.jpg')
defective = glob.glob(r'/kaggle/input/tyre-quality-classification/Digital images of defective and good condition tyres/defective/*.jpg')

random.shuffle(good)
random.shuffle(defective)

print ('number of good tyres : {} images'.format(len (good)))
print ('number of defective tyres : {} images'.format(len (defective)))

In [ ]:
train_good = good [:int (len (good) *0.8)]
test_good = good[int (len (good) *0.8):]

train_defective = defective [:int (len (defective) *0.8)]
test_defective = defective[int (len (defective) *0.8):]

print ('number of train good tyres : {} images'.format(len (train_good)))
print ('number of test good tyres : {} images'.format(len (test_good)))
print ('number of train defective tyres : {} images'.format(len (train_defective)))
print ('number of test defective tyres : {} images'.format(len (test_defective)))

# plot sample of each catogery

In [ ]:
image_shape = 350

def plot_images (data ,image_title,num_of_images):
    plt.figure(figsize=(15,10))
    for i in range (num_of_images):
        random_image = random.choice(data)
        if type (data) == list:
            random_image = cv2.resize(cv2.imread(image), (image_shape, image_shape) )
        plt.subplot(1,num_of_images,i+1)
        plt.imshow(random_image)
        plt.axis('off')
        plt.title (image_title)
    
    plt.show()

plot_images (train_good ,'Good',5)
plot_images (train_defective ,'Defective',5)

# plot categories distribution

In [ ]:
def plt_pie (data,labels,colors,title):
    plt.figure(figsize=(20,10))
    my_circle=plt.Circle( (0,0), 0.7, color='white')
    plt.pie(data,labels = labels, colors = colors , autopct='%1.1f%%')
    p=plt.gcf()
    p.gca().add_artist(my_circle)
    plt.title (title,fontsize = 22)
    plt.show()
    
data = [len (train_good) , len (train_defective) ]
labels = ['Good' , 'Defective' ]

plt_pie (data,labels,['blue','red'] , 'Categories distribution')

# define function to ColorJitter images

In [ ]:
def colorjitter(img, cj_type="b"):
    '''
    ### Different Color Jitter ###
    img: image
    cj_type: {b: brightness, s: saturation, c: constast}
    '''
    images =[]
    if cj_type == "b":
        value = np.random.choice(np.array([-50, -40, -30, 30, 40, 50]))
        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        h, s, v = cv2.split(hsv)
        if value >= 0:
            lim = 255 - value
            v[v > lim] = 255
            v[v <= lim] += value
        else:
            lim = np.absolute(value)
            v[v < lim] = 0
            v[v >= lim] -= np.absolute(value)

        final_hsv = cv2.merge((h, s, v))
        img = cv2.cvtColor(final_hsv, cv2.COLOR_HSV2BGR)
        return (img)
    elif cj_type == "s":
        value = np.random.choice(np.array([-50, -40, -30, 30, 40, 50]))
        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        h, s, v = cv2.split(hsv)
        if value >= 0:
            lim = 255 - value
            s[s > lim] = 255
            s[s <= lim] += value
        else:
            lim = np.absolute(value)
            s[s < lim] = 0
            s[s >= lim] -= np.absolute(value)

        final_hsv = cv2.merge((h, s, v))
        img = cv2.cvtColor(final_hsv, cv2.COLOR_HSV2BGR)
        return (img)
    elif cj_type == "c":
        brightness = 10
        contrast = random.randint(40, 100)
        dummy = np.int16(img)
        dummy = dummy * (contrast/127+1) - contrast + brightness
        dummy = np.clip(dummy, 0, 255)
        img = np.uint8(dummy) 
        return (img)

In [ ]:
def random_flip (img):
    img = cv2.flip (img,random.choice([0,1]))  #1 ---- vertical flip  0----- horz flip
    return img

# increase images by  colorjitter

In [ ]:
def increase_image (data, target):
    images =[]
    for image in data:
        img = cv2.resize(cv2.imread(image), (image_shape, image_shape) )
        images.append(img)
    for i in range (int (target / len (data))):
        for image in data:
            img = cv2.resize(cv2.imread(image), (image_shape, image_shape) )
            img = random_flip (img)
            images.append (colorjitter(img,random.choice(['b','s','c']) ))
            if (len (images)>= target ):
                break
    return np.array (images)

In [ ]:
def convert_links_to_images(data):
    images =[]
    for image in data:
        img = cv2.resize(cv2.imread(image), (image_shape, image_shape) )
        images.append(img)
    return np.array (images)

In [ ]:
train_good_images = increase_image (train_good, 1250)
train_defective_images = increase_image (train_defective, 1250)
test_good_images = convert_links_to_images(test_good)
test_defective_images = convert_links_to_images(test_defective)

# plot distribution of train data after balncing

In [ ]:
plt_pie ([len (train_good_images) ,len (train_defective_images)] ,
         labels,['blue','red'],'Train distribution after balncing data')

# plot sample from each category after editing images

In [ ]:
plot_images (train_good_images ,'Normal',5)
plot_images (train_defective_images ,'Pneumonia',5)

# get train and test data

In [ ]:
y_train = np.concatenate ([np.zeros (len(train_good_images)) , np.ones (len (train_defective_images)) ])
y_test = np.concatenate ([np.zeros (len(test_good_images)) , np.ones (len (test_defective_images))] )

X_train = np.concatenate ([train_good_images,train_defective_images],axis = 0)
X_test = np.concatenate ([test_good_images,test_defective_images],axis = 0)

print (X_train.shape)
print (y_train.shape)
print (X_test.shape)
print (y_test.shape)
#------------------------free some memory---------------
train_good_images,train_defective_images,test_good_images,test_defective_images = None,None,None,None
del train_good_images,train_defective_images,test_good_images,test_defective_images 

# create datagenerator for train and for test

In [ ]:
train_datagen = ImageDataGenerator( rescale=.1/255,
                                    rotation_range=40,
                                    width_shift_range=0.2,
                                    height_shift_range=0.2,
                                    shear_range=0.2,
                                    zoom_range=0.2,
                                    horizontal_flip=True,
                                    vertical_flip=True,
                                    fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=.1/255,)

train_generator = train_datagen.flow(X_train, y_train,batch_size=16,shuffle=True)
test_generator = test_datagen.flow(X_test, y_test,batch_size=16,shuffle=False)
#---------------------------free some memory--------------------
X_train,y_train,X_test = None, None,None
del X_train,y_train,X_test

# get inception_v3 pre trained model

In [ ]:
inception = keras.applications.inception_v3.InceptionV3  (input_shape = (image_shape, image_shape,3),
                                                          include_top = False,
                                                          weights = 'imagenet')
inception.trainable = False         
last_output = inception.get_layer('mixed8').output 

# build full-model and fit model

In [ ]:
x = Flatten()(last_output)
x = Dense(32, activation='relu')(x)
x = BatchNormalization()(x)
x = Dense(64, activation='relu')(x)
x = BatchNormalization()(x)
x = Dense(1, activation='sigmoid')(x)

model = Model(inputs=inception.input, outputs=x)

callbacks = [EarlyStopping(monitor='val_loss', patience=10, restore_best_weights = True,verbose = 1),
             ModelCheckpoint(filepath='model.h5', monitor='val_loss', save_best_only=True)]
            
model.compile(optimizer=keras.optimizers.Adam(learning_rate=5e-4),
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(train_generator,epochs=100,validation_data=test_generator,callbacks=[callbacks])
model.load_weights('model.h5')

# plot loss and accuracy

In [ ]:
train_loss = history.history['loss']
val_loss = history.history['val_loss']
tarin_acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

plt.plot(range (len (train_loss)),train_loss,c='b',label = 'training loss')
plt.plot (range (len (val_loss)),val_loss,c='r',label = 'validation loss')
plt.title ('model loss',loc ='right')
plt.legend()
plt.show()

plt.plot(range (len (tarin_acc)),tarin_acc,c='b',label = 'training accuracy')
plt.plot (range (len (val_acc)),val_acc,c='r',label = 'validation accuracy')
plt.title ('model accuracy',loc ='right')
plt.legend()
plt.show()

# get predicted output

In [ ]:
y_pred = model.predict (test_generator)
y_pred = y_pred.reshape (-1,)
y_pred = np.float16(y_pred>=0.5 )

# print confusion matrix and classification report

In [ ]:
sns.heatmap(confusion_matrix(y_test,y_pred),annot = True,fmt='d')
print (classification_report (y_test,y_pred))